In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

# Part 1 - Data

## Let's Load in the image data for all the pieces
#### * While also tracking the class of each image in a separate array *
#### Note that images are all different sizes. There are smart ways to resize images. This includes composing multiple transforms, like scaling down certain axes, then cropping, etc. For the purpose of this project I will simply resize all images to 256x256 RGB images.

In [14]:
# All piece names in Chess
pieces = np.array(["Bishop", "King", "Knight", "Pawn", "Queen", "Rook"])

# Store our images
image_data = []
# Store their labels
image_labels = np.array([])

fp = "data/"
for piece in pieces:
    
    # Directory of images for this piece
    piece_dir = fp + piece
    
    # Get all image names in this dir 
    piece_images = os.listdir(piece_dir)

    # For each image name
    for image in piece_images:
        
        # Ignore .gif files
        if (image.split(".")[1] != "gif"):
        
            # Create the filepath
            image_fp = fp + piece + "/" + image

            # Open and resize the image
            img = np.array(Image.open(image_fp).resize((256,256)))

            image_data.append(img)
            image_labels = np.append(image_labels, piece)
        
image_data = np.array(image_data)

In [16]:
image_data.shape

(555, 256, 256, 3)